In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense
import os

2025-05-20 08:32:08.111521: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 08:32:08.156801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747701128.183054  554071 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747701128.189294  554071 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747701128.224559  554071 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

#### Data Preparation

In [2]:
# Create a dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    "data/cub_200/images",
    labels='inferred',            # Automatically infer labels from subdirectory names
    label_mode='int',             # 'int', 'categorical', or None
    image_size=(224, 224),        # Resize images to this size
    batch_size=64,                # Batch size
    shuffle=True                  # Shuffle the data
)

Found 11788 files belonging to 200 classes.


I0000 00:00:1747701243.079701  554071 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13553 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
# Check the dataset
for images, labels in dataset.take(1):
    print(images.shape)  # (64, 224, 224, 3)
    print(labels)

(64, 224, 224, 3)
tf.Tensor(
[169 103 137 197 156  87 185 136 153 168   9  32  91 144   7  34  41 111
 124 121  83 156 109 137 139  74  10   3 163 132  22 156 139 102  86 132
  68 162  32  85  38  41  68  67 160 113 196   1 107 174  38  50  46  68
 147 159 117 152 116 175 125  66 149  81], shape=(64,), dtype=int32)


2025-05-20 08:34:06.931866: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# Define the augmentation layer
data_augmentation = Sequential([
    keras.layers.RandomFlip("horizontal"),  # Random horizontal flipping
    keras.layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)),  # Random scaling
    keras.layers.RandomContrast(factor=0.2),  # Random contrast adjustment
    keras.layers.RandomBrightness(factor=0.2)  # Random brightness adjustment
])

In [5]:
# Apply augmentations to a dataset
def preprocess_dataset(image, label):
    image = data_augmentation(image)

    return image, label

In [6]:
# Prepare the dataset
augmented_dataset = dataset.map(preprocess_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [7]:
# Get dataset size
dataset_size = len(dataset)
train_size = int(0.5 * dataset_size)  # 50% for training
val_size = dataset_size - train_size  # 50% for validation

train_ds = dataset.take(train_size)  # First 50%
val_ds = dataset.skip(train_size)  # Remaining 50%

# Prepare datasets
# train_ds = train_ds.shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
# val_ds = val_ds.batch(64).prefetch(tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)


In [8]:
for image, label in train_ds.take(1):
    print(image.shape)  # Should be [batch_size, height, width, channels].
    print(label.shape)  # Should be [batch_size].

(64, 224, 224, 3)
(64,)


2025-05-20 08:34:26.001702: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Model Creation

In [9]:
base_model = keras.applications.ResNet101V2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

In [10]:
# Freeze the base_model
base_model.trainable = True
# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))
# Pre-trained weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
# scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
# x = scale_layer(inputs)
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(inputs, training=True)
x = keras.layers.GlobalAveragePooling2D()(x)
# x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
# outputs = keras.layers.Dense(1)(x)

# base_model is running in inference mode here.
outputs = keras.layers.Dense(200, activation="softmax", name="Predictions")(x)
new_model = Model(inputs, outputs)

new_model.summary(show_trainable=True)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_2 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ resnet101v2 (Functional)    │ (None, 7, 7, 2048)    │ 42,626,560 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d    │ (None, 2048)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Predictions (Dense)         │ (None, 200)           │    409,800 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 43,036,360 (164.17 MB)

 Trainable params: 42,938,696 (163.80 MB)

 Non-trainable params: 97,664 (381.50 KB)

In [11]:
def compile_model(model, lr=0.01, m=0.9, wd=0.001):
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr,
                                                             decay_steps=940,
                                                             decay_rate=0.1,)

    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule,
                                      momentum=m,
                                      weight_decay=wd),
        # loss=keras.losses.BinaryCrossentropy,
        loss="sparse_categorical_crossentropy",
        # metrics=[keras.metrics.BinaryAccuracy],
        metrics=["sparse_categorical_accuracy"]
    )
    return model

### Model Fitting

In [12]:
learning_rates = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]
momentums = [0.0, 0.8, 0.9, 0.95, 0.99]

In [13]:
# Prepare to log results
results = []

# Create directories for saving
base_checkpoint_dir = "checkpoints_momentum"
base_log_dir = "logs_momentum"
os.makedirs(base_checkpoint_dir, exist_ok=True)
os.makedirs(base_log_dir, exist_ok=True)

# Loop through each momentum value
for m in momentums:
    print(f"Training model with momentum: {m}")
    # Build the model
    model = compile_model(new_model, m=m, wd=0.0)

    # Define callbacks
    checkpoint_dir = os.path.join(base_checkpoint_dir, f"momentum_{m}")
    log_dir = os.path.join(base_log_dir, f"momentum_{m}")
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)

    callbacks = [
        keras.callbacks.ModelCheckpoint(
            # Path where to save the model
            # The two parameters below mean that we will overwrite
            # the current checkpoint if and only if
            # the `val_loss` score has improved.
            # The saved model name will include the current epoch.
            filepath=os.path.join(checkpoint_dir,
                                  "model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras"),
            save_best_only=True,  # Only save a model if `val_loss` has improved.
            monitor="val_loss", # Monitor validation loss
            mode="min",           # Save when val_loss decreases
            verbose=1,),    # Print when saving

        keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,  # How often to log histogram visualizations
        embeddings_freq=0,  # How often to log embedding visualizations
        update_freq="epoch",    # How often to write logs (default: once per epoch)
        write_graph=True,      # Save the computational graph
        write_images=True,      # Log model input images (if applicable)
        )
    ]

    # Train the model
    history = model.fit(train_ds,
                        epochs=30,
                        # steps_per_epoch=94,
                        validation_data=val_ds,
                        callbacks=callbacks)

    # Record results
    best_val_loss = min(history.history["val_loss"])
    best_val_accuracy = max(history.history["val_sparse_categorical_accuracy"])
    results.append({"momentum": m, "val_loss": best_val_loss, "val_accuracy": best_val_accuracy})

# Save results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("momentum_tuning_results.csv", index=False)
print("Results saved to momentum_tuning_results.csv")

Training model with momentum: 0.0
Epoch 1/30


I0000 00:00:1747701323.289459  556183 service.cc:152] XLA service 0x7f298c0030d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747701323.289504  556183 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti SUPER, Compute Capability 8.9
2025-05-20 08:35:23.948793: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747701325.933254  556183 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-20 08:35:27.952105: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22697_0', 328 bytes spill stores, 328 bytes spill loads

2025-05-20 08:35:28.040853: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fu

92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - loss: 5.3048 - sparse_categorical_accuracy: 0.0216

2025-05-20 08:36:13.436689: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2964', 8 bytes spill stores, 8 bytes spill loads

2025-05-20 08:36:13.605162: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2964', 12 bytes spill stores, 12 bytes spill loads

2025-05-20 08:36:13.774677: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2964', 120 bytes spill stores, 120 bytes spill loads

2025-05-20 08:36:14.022364: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2964', 3972 bytes spill stores, 3952 bytes spill loads

2025-05-20 08:36:14.083531: I extern


Epoch 1: val_loss improved from inf to 4.40653, saving model to checkpoints_momentum/momentum_0.0/model_epoch_01_val_loss_4.41.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 79s 437ms/step - loss: 5.3018 - sparse_categorical_accuracy: 0.0220 - val_loss: 4.4065 - val_sparse_categorical_accuracy: 0.1031
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - loss: 3.5153 - sparse_categorical_accuracy: 0.3596
Epoch 2: val_loss improved from 4.40653 to 3.06598, saving model to checkpoints_momentum/momentum_0.0/model_epoch_02_val_loss_3.07.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 35s 348ms/step - loss: 3.5135 - sparse_categorical_accuracy: 0.3599 - val_loss: 3.0660 - val_sparse_categorical_accuracy: 0.2941
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - loss: 2.3262 - sparse_categorical_accuracy: 0.6664
Epoch 3: val_loss improved from 3.06598 to 2.66759, saving model to checkpoints_momentum/momentum_0.0/model_epoch_03_val_loss_2.67.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 35s 344ms/step - loss: 2.3252 - sparse_cate

In [14]:
results_df

,momentum,val_loss,val_accuracy
0,0.00,2.167397,0.498136
1,0.80,1.472176,0.623898
2,0.90,1.482276,0.625763
3,0.95,1.584263,0.620339
4,0.99,1.679947,0.606949


In [25]:
def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['sparse_categorical_accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

In [21]:
# Compute Top-1 Validation Error
val_accuracy = history.history['val_sparse_categorical_accuracy']
val_error = [1 - acc for acc in val_accuracy]

min_val_error = min(val_error)
best_epoch = val_error.index(min_val_error) + 1

print(f"Minimal Top-1 Validation Error: {min_val_error:.4f} at Epoch {best_epoch}")

Minimal Top-1 Validation Error: 0.3783 at Epoch 1


In [16]:
# Plot Validation Error
plt.plot(range(1, len(val_error) + 1), val_error, label='Top-1 Validation Error')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.legend()
plt.show()

NameError: name 'val_error' is not defined

In [15]:
plot_loss_accuracy(history)

NameError: name 'plot_loss_accuracy' is not defined